# Bussiness Problem

## Imports

In [1]:
import psycopg2
import numpy as np
import pandas as pd
import postgres_crud as pc

## Helper Functions

# Coletando os Dados

## Conhecendo as tabelas do BD.

In [12]:
# Parametros
HOST = "comunidade-ds-postgres.c50pcakiuwi3.us-east-1.rds.amazonaws.com"
PORT = "5432"
DATABASE = "comunidadedsdb"
USERNAME = "member"
PASSWORD = "cdspa"

# Query SQL to find the tables names
query =   """
          SELECT table_name 
          FROM information_schema.tables
          WHERE table_schema='pa004'
          AND table_type='BASE TABLE';
          
          """

data = pc.Postgresql.retrieve_data(query=query,
                                   user=USERNAME,
                                   password=PASSWORD,
                                   host=HOST,
                                   port=PORT,
                                   database=DATABASE,
                                   connect=True,
                                   objeto = 'pd')

data

Buscando os dados!!!
Conexão com Postgresql fechada


,table_name
0,users
1,vehicle
2,insurance


Cool we have three tables to work with 'users', 'vehicle', 'insurance'. Let's get them individually.

In [29]:
# querys
query_insurance = """SELECT * FROM pa004.insurance; """
query_users = """SELECT * FROM pa004.users; """
query_vehicle = """SELECT * FROM pa004.vehicle; """

QUERY_LIST = [query_insurance,query_users, query_vehicle]

# Generating a list of dataframes
dataframes = []
for query in QUERY_LIST:
    
    data = pc.Postgresql.retrieve_data(query=query,
                                   user=USERNAME,
                                   password=PASSWORD,
                                   host=HOST,
                                   port=PORT,
                                   database=DATABASE,
                                   connect=True,
                                   objeto = 'pd')
    dataframes.append(data)
    

# Checking duplicated ids
print("\nThere is duplicated ids ?")
for i in range(0,3):
    df = dataframes[i]
    dp = df[['id']].duplicated()
    result = dp.unique()
    print(f'\nDuplicated ids in df {i} - {result}')

Buscando os dados!!!
Conexão com Postgresql fechada
Buscando os dados!!!
Conexão com Postgresql fechada
Buscando os dados!!!
Conexão com Postgresql fechada

There is duplicated ids ?

Duplicated ids in df 0 - [False]

Duplicated ids in df 1 - [False]

Duplicated ids in df 2 - [False]


In [27]:
# Splitting the dataframes in differents DFs.

# Insurance df
insurance_df = dataframes[0]
# Users df
users_df = dataframes[1]
# Vehicle df
vehicles_df = dataframes[2]

# Logical Test - we want a empty 
diff = set(insurance_df['id']).difference(set(users_df['id']), set(vehicles_df['id']))
print(diff)

set()


In [32]:
dataframes[2].head()

,id,driving_license,vehicle_age,vehicle_damage
0,1,1,> 2 Years,Yes
1,2,1,1-2 Year,No
2,3,1,> 2 Years,Yes
3,4,1,< 1 Year,No
4,5,1,< 1 Year,No


## Retriving all data with one Query

In [36]:
# Query SQL to find the tables names
query =   """
            SELECT * 
            FROM pa004.insurance i
            INNER JOIN pa004.users u
                 on i.id = u.id 
            INNER JOIN pa004.vehicle v
                 on u.id = v.id
          """

data = pc.Postgresql.retrieve_data(query=query,
                                   user=USERNAME,
                                   password=PASSWORD,
                                   host=HOST,
                                   port=PORT,
                                   database=DATABASE,
                                   connect=True,
                                   objeto = 'pd')

data.head()

Buscando os dados!!!
Conexão com Postgresql fechada


,id,previously_insured,annual_premium,vintage,response,id,gender,age,region_code,policy_sales_channel,id,driving_license,vehicle_age,vehicle_damage
0,7,0,23367.0,249,0,7,Male,23,11.0,152.0,7,1,< 1 Year,Yes
1,13,1,31409.0,221,0,13,Female,41,15.0,14.0,13,1,1-2 Year,No
2,18,1,46622.0,299,0,18,Female,25,35.0,152.0,18,1,< 1 Year,No
3,31,0,2630.0,136,0,31,Female,26,8.0,160.0,31,1,< 1 Year,No
4,39,0,42297.0,264,0,39,Male,45,8.0,124.0,39,1,1-2 Year,Yes


# Data Description